The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity $ \omega $ with the trapezoid rule as:

\begin{equation}
\mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}| \omega(\mathbf{x}(t)) - \overline{\omega}(t) |dt \approx 
\dfrac{1}{t_N-t_0}\left[\frac{|\omega(\mathbf{x}(t_0)) - \overline{\omega}(t_0) | \Delta t }{2} + \frac{|\omega(\mathbf{x}(t_N)) - \overline{\omega}(t_N) | \Delta t }{2} + \sum_{i = 1}^{N-1}| \omega(\mathbf{x}(t_i)) - \overline{\omega}(t_i) | \Delta t \right] \tag{1}
\end{equation}

where $ \omega(\mathbf{x}(t_i)) $ indicates the vorticity at $ \mathbf{x}(t_i) $. $ \overline{\omega}(t_i) $ is the spatial average of the vorticity at time $ t_i $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| omega | array (Nt, 3, Npoints) | vorticity $ \omega $ along trajectories|
| times | array (Nt, )| time array, equi spaced array is assumed|
| omega_avg | array (Nt, )| spatial average of the vorticity. If None, the function calculates this. |
| LAVD | array (Npoints) | $ \mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) $|

In [9]:
# import sys, os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])
# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [ ]:
# Import numpy
import numpy as np

# Import function which calculates gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# import math tools
from math import sqrt

# import progress bar
from tqdm.notebook import tqdm

In [14]:
def _LAVD(omega, times, omega_avg = None):
    ''' 
    The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity with the trapezoid rule.
    Integrate the absolute deviation of the vorticity from its spatial mean along trajectories
    and divide by the length of the time interval.
    
    Parameters:
        omega: array(Nt, 3, Npoints), the vorticity vector computed along trajectories
        times: array(Nt, ), time array. Uniform spacing is assumed
        omega_avg: array(Nt,), spatial average of the vorticity for each time instant
        
    Returns:
        LAVD: array(Npoints,), integrated |\omega - average(\omega)| / t_N - t_0, the LAVD field
    '''
    
    omega = omega.reshape((omega.shape[0], 3, -1))
    lenT = times[-1] - times[0] # calculate length of time interval
    dt = times[1] - times[0] # assume uniform dt
    
    # Compute averaged vorticity if not specified in the args.
    if omega_avg is None:
        omega_avg = [] # list (Nt,)
        for t in range(omega.shape[0]):
            omega_avg.append(np.mean(omega[t,:,:], axis = -1))

    # Compute LAVD
    omega_avg = np.array(omega_avg)
    LAVD = np.zeros((omega.shape[2]))
    omega_dif = (omega[0,:,:].T - omega_avg[0,:]).T # 0 th step in the trapezoid rule
    LAVD += np.sqrt(omega_dif[0]**2 + omega_dif[1]**2 + omega_dif[2]**2) * dt / 2

    for t in tqdm(range(1,omega.shape[0]-1)): # integrate with the trapezoid rule
        omega_dif = (omega[t,:,:].T - omega_avg[t,:]).T
        LAVD += np.sqrt(omega_dif[0]**2+omega_dif[1]**2+omega_dif[2]**2) * dt
            
    indexN = omega.shape[0]-1
    omega_dif = (omega[indexN,:,:].T - omega_avg[indexN,:]).T # N th step in the trapezoid rule
    LAVD += np.sqrt(omega_dif[0]**2 + omega_dif[1]**2 + omega_dif[2]**2) * dt / 2
    
    return LAVD / lenT